# Домашнее задание  № 5. Матричные разложения/Тематическое моделирование

In [11]:
import gensim
import pandas as pd
import numpy as np
from pymorphy2 import MorphAnalyzer
import pyLDAvis
from collections import Counter
from string import punctuation
from razdel import tokenize as razdel_tokenize
from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.decomposition import TruncatedSVD, NMF, PCA, LatentDirichletAllocation
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
import warnings
from matplotlib import pyplot as plt
import seaborn as sns
morph = MorphAnalyzer()
warnings.filterwarnings("ignore")

In [12]:
# добавим лемматизацию
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    normalized_text = [morph.parse(word)[0].normal_form for word in normalized_text]
    return ' '.join(normalized_text)

### Задание № 1 (4 балла)

Попробуйте матричные разложения с 5 классификаторами - SGDClassifier, KNeighborsClassifier,  RandomForest, ExtraTreesClassifier (про него подробнее почитайте в документации, он похож на RF). Используйте и NMF и SVD. Сравните результаты на кросс-валидации и выберите лучшее сочетание.

В итоге у вас должно получиться, как минимум 10 моделей (два разложения на каждый классификатор). Используйте 1 и те же параметры кросс-валидации. Параметры векторизации, параметры K в матричных разложениях, параметры классификаторов могут быть разными между экспериментами.

Можете взять поменьше данных, если все будет обучаться слишком долго (не ставьте параметр K слишком большим в NMF, иначе точно будет слишком долго)

In [ ]:
data = pd.read_csv('avito_category_classification.csv')
data['description_norm'] = data['description'].apply(normalize)

In [3]:
def eval_table(X, y, pipeline, N=6):
    labels = list(set(y))
    fold_metrics = pd.DataFrame(index=labels)
    errors = np.zeros((len(labels), len(labels)))
    kfold = StratifiedKFold(n_splits=N, shuffle=True, )
    
    for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
        pipeline.fit(X[train_index], y[train_index])
        preds = pipeline.predict(X[test_index])
        
        # записываем метрику и индекс фолда
        fold_metrics[f'precision_{i}'] = precision_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'recall_{i}'] = recall_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'f1_{i}'] = f1_score(y[test_index], preds, labels=labels, average=None)
        errors += confusion_matrix(y[test_index], preds, labels=labels, normalize='true')

    result = pd.DataFrame(index=labels)
    result['precision'] = fold_metrics[[f'precision_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['precision_std'] = fold_metrics[[f'precision_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['recall'] = fold_metrics[[f'recall_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['recall_std'] = fold_metrics[[f'recall_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['f1'] = fold_metrics[[f'f1_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['f1_std'] = fold_metrics[[f'f1_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result.loc['mean'] = result.mean().round(2)
    errors /= N
    
    return result, errors

In [4]:
pipeline_svd_RF = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', TruncatedSVD(500)),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=10))
])
pipeline_svd_SGDC = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', TruncatedSVD(500)),
    ('clf', SGDClassifier(max_iter=1000, tol=1e-3))
])
pipeline_svd_KNC = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', TruncatedSVD(500)),
    ('clf', KNeighborsClassifier())
])
pipeline_svd_ETC = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', TruncatedSVD(500)),
    ('clf', ExtraTreesClassifier(n_estimators=100, max_depth=10))
])
svd = {'Random Forest': pipeline_svd_RF, 'SGDClassifier': pipeline_svd_SGDC, 'KNeighborsClassifier': pipeline_svd_KNC,'ExtraTreesClassifier': pipeline_svd_ETC}

In [5]:
pipeline_nmf_RF = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', NMF(100)),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=10))
])
pipeline_nmf_SGDC = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', NMF(100)),
    ('clf', SGDClassifier(max_iter=1000, tol=1e-3))
])
pipeline_nmf_KNC = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', NMF(100)),
    ('clf', KNeighborsClassifier())
])
pipeline_nmf_ETC = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', NMF(100)),
    ('clf', ExtraTreesClassifier(n_estimators=100, max_depth=10))
])
nmf = {'Random Forest': pipeline_nmf_RF, 'SGDClassifier': pipeline_nmf_SGDC, 'KNeighborsClassifier': pipeline_nmf_KNC,'ExtraTreesClassifier': pipeline_nmf_ETC}

In [7]:
print('svd: ')
for name, pipe in svd.items():
    result, error = eval_table(data['description_norm'], data['category_name'], pipe)
    print(name, result.loc["mean"]["f1"].round(3))
print('nmf: ')
for name, pipe in nmf.items():
    result, error = eval_table(data['description_norm'], data['category_name'], pipe)
    print(name, result.loc["mean"]["f1"].round(3))    

svd: 
Random Forest 0.45
SGDClassifier 0.73
KNeighborsClassifier 0.46
ExtraTreesClassifier 0.24
nmf: 
Random Forest 0.55
SGDClassifier 0.52
KNeighborsClassifier 0.42
ExtraTreesClassifier 0.37


По f1 лучшее сочетание - это SVD+SGD

### Задание № 2 (6 баллов)

В Gensim тоже можно добавить нграммы и tfidf. Постройте 1 модель без них (как в семинаре) и еще 3 модели (1 с нграммами, 1 с tfidf и 1 с нграммами и с tfidf). Сранивте качество с помощью метрик (перплексия, когерентность) и на глаз. Определите лучшую модель. Для каждой модели выберите 1 самую красивую на ваш взгляд тему.

Используйте данные википедии из семинара. Можете взять поменьше данных, если все обучается долго.

Важное требование - получившиеся модели не должны быть совсем плохими. Если хороших тем не получается, попробуйте настроить гиперпараметры, отфильтровать словарь по-другому. 

Нграммы добавляются вот так (перед созданиеv словаря)

In [34]:
texts = open('wiki_data.txt', encoding="utf8").read().splitlines()[:1000]
texts = ([normalize(text) for text in texts])

texts = [text.split() for text in texts]
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts] 

# ! не забудьте, что далее вам нужно будет использовать ngrammed_texts

!! В модели с нграммами вначале посмотрите, что получается после преобразования
Если вы выведите несколько первых текстов в ngrammed_texts, то там должно быть что-то такое:

In [20]:
num_topics = 100

In [29]:
def metrics(model, texts, dictionary, corpus):
    topics = []
    for topic_id, topic in model.show_topics(num_topics=num_topics, formatted=False):
        topic = [word for word, _ in topic]
        topics.append(topic)
    coherence_model_lda = gensim.models.CoherenceModel(topics=topics,
                                                       texts=texts,
                                                       dictionary=dictionary, coherence='c_v')
    return coherence_model_lda.get_coherence(), np.exp2(-model.log_perplexity(corpus))


In [30]:
dictionary = gensim.corpora.Dictionary((text for text in texts))
dictionary.filter_extremes(no_above=0.1, no_below=10)
dictionary.compactify()

corpus = [dictionary.doc2bow(text) for text in texts]

lda_1 = gensim.models.LdaMulticore(corpus, 
                                 num_topics, 
                                 alpha='asymmetric',
                                 id2word=dictionary, 
                                 passes=10)
coherence, perplexity = metrics(lda_1, texts, dictionary, corpus)
print('Default')
print('Coherence: ',coherence, 'Perplexity: ',perplexity)


Default
Coherence:  0.4972254817989352 Perplexity:  220.5182706399927


In [41]:
lda_1.print_topics()[:15]

[(99,
  '0.058*"посёлок" + 0.025*"житель" + 0.022*"село" + 0.018*"округа" + 0.018*"иметься" + 0.017*"ботсвана" + 0.016*"пункт" + 0.015*"море" + 0.013*"центральный" + 0.012*"данные"'),
 (98,
  '0.053*"животное" + 0.028*"процесс" + 0.022*"задача" + 0.021*"способность" + 0.017*"связь" + 0.016*"использовать" + 0.015*"учёный" + 0.013*"тип" + 0.013*"способный" + 0.012*"исследование"'),
 (97,
  '0.052*"хутор" + 0.031*"дубовский" + 0.026*"поселение" + 0.024*"сельский" + 0.021*"газета" + 0.014*"численность" + 0.013*"ростовский" + 0.013*"иметься" + 0.013*"река" + 0.013*"согласно"'),
 (96,
  '0.049*"клуб" + 0.042*"команда" + 0.032*"матч" + 0.030*"чемпионат" + 0.021*"сезон" + 0.015*"кубок" + 0.015*"чемпион" + 0.014*"контракт" + 0.014*"против" + 0.014*"выступать"'),
 (95,
  '0.045*"тысяча" + 0.018*"южный" + 0.018*"данные" + 0.014*"2015" + 0.013*"колледж" + 0.012*"перепись" + 0.011*"университет" + 0.009*"библиотека" + 0.009*"здание" + 0.009*"житель"'),
 (94,
  '0.029*"культура" + 0.019*"сср" + 0.018

мне нравится эта тема, понятно, что она про какой-то вид спорта:<br/>
'0.049*"клуб" + 0.042*"команда" + 0.032*"матч" + 0.030*"чемпионат" + 0.021*"сезон" + 0.015*"кубок" + 0.015*"чемпион" + 0.014*"контракт" + 0.014*"против" + 0.014*"выступать"'<br/>
здесь про образование:<br/>
'0.041*"институт" + 0.027*"наука" + 0.023*"научный" + 0.021*"профессор" + 0.019*"метод" + 0.019*"академия" + 0.017*"определение" + 0.016*"исследование" + 0.014*"наш" + 0.013*"кафедра"<br/>
про кино: <br/>
'0.063*"фильм" + 0.030*"роль" + 0.025*"’" + 0.020*"ямайка" + 0.011*"сыграть" + 0.011*"главный" + 0.010*"американский" + 0.009*"1948" + 0.009*"актёр" + 0.009*"сняться"'<br/>

In [31]:
dictionary = gensim.corpora.Dictionary((text for text in texts))
dictionary.filter_extremes(no_above=0.1, no_below=10)
dictionary.compactify()

corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
tf_idf_corpus = tfidf[corpus]

lda_2 = gensim.models.LdaMulticore(tf_idf_corpus,
                                        num_topics,
                                        alpha='asymmetric',
                                        id2word=dictionary,
                                        passes=10)
coherence, perplexity = metrics(lda_2, texts, dictionary, corpus)
print('TF-idf')
print('Coherence: ',coherence, 'Perplexity: ',perplexity)

TF-idf
Coherence:  0.4531671817773997 Perplexity:  515.2402667594639


In [43]:
lda_2.print_topics()[:15]

[(99,
  '0.000*"личный" + 0.000*"мария" + 0.000*"историк" + 0.000*"каменный" + 0.000*"конгресс" + 0.000*"конечный" + 0.000*"круглый" + 0.000*"крыло" + 0.000*"золото" + 0.000*"монреаль"'),
 (97,
  '0.120*"фамилия" + 0.000*"сценарий" + 0.000*"анатолий" + 0.000*"российский" + 0.000*"окончить" + 0.000*"москва" + 0.000*"1953" + 0.000*"поиск" + 0.000*"факультет" + 0.000*"руководитель"'),
 (98,
  '0.001*"всемирный" + 0.001*"волость" + 0.001*"промышленность" + 0.001*"горный" + 0.001*"провинция" + 0.001*"уезд" + 0.000*"перевести" + 0.000*"климат" + 0.000*"подчинение" + 0.000*"посёлок"'),
 (96,
  '0.001*"съезд" + 0.001*"1905" + 0.001*"организация" + 0.000*"москва" + 0.000*"партийный" + 0.000*"комитет" + 0.000*"участвовать" + 0.000*"московский" + 0.000*"тюрьма" + 0.000*"член"'),
 (95,
  '0.001*"мост" + 0.001*"рабочий" + 0.000*"подразделение" + 0.000*"июнь" + 0.000*"рота" + 0.000*"командование" + 0.000*"железнодорожный" + 0.000*"армия" + 0.000*"корпус" + 0.000*"бой"'),
 (94,
  '0.001*"сборный" + 0

мне кажется, тут про ссср:<br/>
'0.001*"съезд" + 0.001*"1905" + 0.001*"организация" + 0.000*"москва" + 0.000*"партийный" + 0.000*"комитет" + 0.000*"участвоват" + 0.000*"московский" + 0.000*"тюрьма" + 0.000*"член" <br/>
а тут про спорт:<br/>
'0.001*"сборный" + 0.001*"ссср" + 0.001*"тренер" + 0.001*"матч" + 0.001*"чемпионат" + 0.000*"победа" + 0.000*"чемпион" + 0.000*"заслужить" + 0.000*"москва" + 0.000*"спортивный"'<br/>

In [39]:
dictionary = gensim.corpora.Dictionary((text for text in ngrammed_texts))
dictionary.filter_extremes(no_above=0.05, no_below=10)
dictionary.compactify()

corpus = [dictionary.doc2bow(text) for text in ngrammed_texts]

lda_3 = gensim.models.LdaMulticore(corpus, 
                                     num_topics, 
                                     alpha='asymmetric',
                                     id2word=dictionary, 
                                     passes=10) 
coherence, perplexity = metrics(lda_3, ngrammed_texts, dictionary, corpus)
print('ngrams')
print('Coherence: ',coherence, 'Perplexity: ',perplexity)

ngrams
Coherence:  0.45483407229482187 Perplexity:  264.19025257345805


In [44]:
lda_3.print_topics()[:15]

[(99,
  '0.045*"церковь" + 0.021*"1965_год" + 0.017*"здание" + 0.016*"автомобиль" + 0.014*"процесс" + 0.012*"московский" + 0.011*"задача" + 0.011*"архитектор" + 0.011*"тренер" + 0.011*"дерево"'),
 (98,
  '0.077*"станция" + 0.023*"комплекс" + 0.021*"институт" + 0.015*"наш" + 0.014*"опытный" + 0.013*"федеральный" + 0.013*"академик" + 0.012*"совет" + 0.012*"передавать" + 0.012*"оборудование"'),
 (96,
  '0.039*"монастырь" + 0.024*"собор" + 0.021*"культура" + 0.014*"святой" + 0.013*"башня" + 0.012*"регион" + 0.011*"озеро" + 0.011*"восточный" + 0.010*"э" + 0.010*"до_наш"'),
 (97,
  '0.022*"фестиваль" + 0.021*"выставка" + 0.020*"городской" + 0.020*"пространство" + 0.018*"мы" + 0.011*"среда" + 0.011*"площадь" + 0.011*"акция" + 0.011*"италия" + 0.010*"архитектор"'),
 (94,
  '0.013*"север" + 0.012*"карта" + 0.009*"22" + 0.009*"канал" + 0.009*"произвести" + 0.009*"китайский" + 0.009*"станция" + 0.009*"перевод" + 0.008*"западный" + 0.008*"запад"'),
 (95,
  '0.150*"уезд" + 0.030*"кнр" + 0.026*"быть

про населенные пункты:<br/>
'0.093*"сесть" + 0.054*"–" + 0.050*"село" + 0.038*"сель" + 0.024*"сельский_поселение" + 0.021*"крым" + 0.017*"километр_к" + 0.015*"поселение" + 0.015*"отделение" + 0.014*"населить_пункт"'<br/>
про урбанистику: <br/>
'0.022*"фестиваль" + 0.021*"выставка" + 0.020*"городской" + 0.020*"пространство" + 0.018*"мы" + 0.011*"среда" + 0.011*"площадь" + 0.011*"акция" + 0.011*"италия" + 0.010*"архитектор"'

In [47]:
dictionary = gensim.corpora.Dictionary((text for text in ngrammed_texts))
dictionary.filter_extremes(no_above=0.05, no_below=10)
dictionary.compactify()

corpus = [dictionary.doc2bow(text) for text in ngrammed_texts]

tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
corpus = tfidf[corpus]

lda_4 = gensim.models.LdaMulticore(corpus, 
                                 15,
                                 alpha='asymmetric',
                                 id2word=dictionary, 
                                 passes=10) 
coherence, perplexity = metrics(lda_4, ngrammed_texts, dictionary, corpus)
print('ngrams + tfidf')
print('Coherence: ',coherence, 'Perplexity: ',perplexity)


ngrams + tfidf
Coherence:  0.35663227044628837 Perplexity:  612.2806054946521


In [48]:
lda_4.print_topics()

[(0,
  '0.003*"сесть" + 0.003*"уезд" + 0.003*"посёлок" + 0.003*"зимний_олимпийский" + 0.002*"клуб" + 0.002*"соревнование_по" + 0.002*"альбом" + 0.002*"сезон" + 0.002*"станция" + 0.002*"турнир"'),
 (1,
  '0.025*"значение" + 0.016*"департамент" + 0.014*"аргентина" + 0.012*"каждый_заезд" + 0.012*"сильнейший_экипаж" + 0.011*"хороший_экипаж" + 0.011*"проходить_несколько" + 0.011*"следующий_раунд" + 0.011*"пункт" + 0.010*"выходить_6"'),
 (2,
  '0.027*"ямайка" + 0.026*"ямайка_принимать" + 0.011*"сборная_страна" + 0.008*"1952" + 0.006*"слава" + 0.006*"два_бронзовый" + 0.006*"шесть_раз" + 0.006*"монастырь" + 0.005*"золотой_медаль" + 0.005*"серебряный_медаль"'),
 (3,
  '0.001*"орудие" + 0.001*"процесс" + 0.000*"институт" + 0.000*"угол" + 0.000*"изображение" + 0.000*"карта" + 0.000*"мм" + 0.000*"параметр" + 0.000*"солнечный" + 0.000*"соревнование"'),
 (4,
  '0.001*"равнина" + 0.001*"запад" + 0.001*"заказ" + 0.001*"гражданский_война" + 0.001*"индеец" + 0.001*"начинаться" + 0.001*"солдат" + 0.001*"

про спортивные состязания:<br/>
'0.000*"чемпионат" + 0.000*"матч" + 0.000*"турнир" + 0.000*"писатель" + 0.000*"1928" + 0.000*"раунд" + 0.000*"каждый_заезд" + 0.000*"победа" + 0.000*"польский" + 0.000*"х"' <br/>


Мне кажется, что модель с нграммами -- самая лучшая и содержит самые осмысленные темы<br/>
по метрикам лучше всего дефолтная модель